In [13]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [14]:
import time
import requests
from datetime import datetime, timedelta
from pathlib import Path
import requests
import json
import os
import pandas as pd

In [15]:
# Load app IDs from file
app_ids = []
with open("app_ids_with_reviews.txt", "r") as f:
    app_ids = [line.strip() for line in f]

# API parameters
base_url = "https://store.steampowered.com/appreviews/"
time_interval = timedelta(hours=24)
end_time = datetime.now()
start_time = datetime(2024, 1, 1, 0, 0, 0)  # Adjust if needed

print(f"Start time: {start_time}")
print(f"End time: {end_time}")
print(start_time.timestamp(), end_time.timestamp())


selected_reviews = []

session = requests.Session()
session.mount("https://", requests.adapters.HTTPAdapter(max_retries=3))

def get_user_reviews(app_id, params, max_retries=3):
    """Fetches reviews from Steam API with retry on timeout."""
    for attempt in range(max_retries):
        try:
            response = session.get(f"{base_url}{app_id}", params=params, timeout=10, verify=True)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.Timeout:
            print(f"Timeout for app {app_id}. Retrying in 10 seconds... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(10)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for app {app_id}: {e}")
            return None  # Stop retrying on non-timeout errors
    return None  # If all retries fail



# # Process each app ID
# for app_id in app_ids:
#     print(f"\nGetting reviews for app_id: {app_id}")
#     params = {
#         "json": 1,
#         "language": "turkish",
#         "cursor": "*",
#         "num_per_page": 100,
#         "filter": "recent",
#     }

#     page = 0
#     passed_start_time = False
#     passed_end_time = False

#     # Get the max 10 pages of reviews for each app if
#     # the reviews are within the time interval
#     # and the reviews are not empty.
#     # If the reviews are empty, break the loop

#     while page < 10 and (not passed_start_time or not passed_end_time):

#         # get the reviews for the app
#         print(f"Getting reviews for app_id: {app_id} page: {page}")
#         reviews_response = get_user_reviews(app_id, params)

#         if not reviews_response or reviews_response.get("success") != 1:
#             print("Request failed or no success response.")
#             break

#         reviews = reviews_response.get("reviews", [])
#         if not reviews:
#             print("Reached the end of the Reviews!!!.")
#             break

#         # While there are still reviews keep extracting them.
#         for review in reviews:
#             timestamp_created = review["timestamp_created"]
#             if timestamp_created > end_time.timestamp():
#                 continue
#             if timestamp_created < start_time.timestamp():
#                 passed_start_time = True
#                 break

#             recommendation_id = review['recommendationid']
#             timestamp_created = review['timestamp_created']
#             timestamp_updated = review['timestamp_updated']
#             # extract the useful (to me) data
#             # Add APP ID to the review 
#             review['app_id'] = app_id
#             author_steamid = review['author']['steamid']        # will automatically redirect to the profileURL if any
#             review_text = review['review']
#             voted_up = review['voted_up']
#             votes_up = review['votes_up']
#             votes_funny = review['votes_funny']
#             weighted_vote_score = review['weighted_vote_score']
#             steam_purchase = review['steam_purchase']
#             received_for_free = review['received_for_free']
#             written_during_early_access = review['written_during_early_access']
            

#             my_review_dict = {
#                 'app_id': app_id,
#                 'recommendationid': recommendation_id,
#                 'author_steamid': author_steamid,

#                 'review_text': review_text,
#                 'timestamp_created': timestamp_created,
#                 'timestamp_updated': timestamp_updated,

#                 'voted_up': voted_up,
#                 'votes_up': votes_up,
#                 'votes_funny': votes_funny,
#                 'weighted_vote_score': weighted_vote_score,
#                 'steam_purchase': steam_purchase,
#                 'received_for_free': received_for_free,
#                 'written_during_early_access': written_during_early_access,
#             }
#             review["app_id"] = app_id
#             selected_reviews.append(review)

#         # Get next page cursor
#         cursor = reviews_response.get("cursor", "")
#         if not cursor:
#             print("Reached the last page!!!")
#             break
        
#         params["cursor"] = cursor
#         print(f"Moving to page {page + 1}. Cursor: {cursor}")
#         page += 1
#         time.sleep(2)

#     time.sleep(2)  # Wait before fetching the next app


# print("Finished processing all app reviews.")

# Process each app ID
for app_id in app_ids:
    print(f"\nGetting reviews for app_id: {app_id}")
    params = {
        "json": 1,
        "language": "turkish",
        "cursor": "*",
        "num_per_page": 100,
        "filter": "recent",
    }

    page = 0
    passed_start_time = False
    passed_end_time = False

    while page < 10 and (not passed_start_time or not passed_end_time):
        print(f"Getting reviews for app_id: {app_id} page: {page}")
        reviews_response = get_user_reviews(app_id, params)

        if not reviews_response or reviews_response.get("success") != 1:
            print("Request failed or no success response.")
            break

        reviews = reviews_response.get("reviews", [])
        if not reviews:
            print("Reached the end of the Reviews!!!.")
            break

        for review in reviews:
            timestamp_created = review["timestamp_created"]
            if timestamp_created > end_time.timestamp():
                continue
            if timestamp_created < start_time.timestamp():
                passed_start_time = True
                break

            # Extract and format the review data
            my_review_dict = {
                'app_id': app_id,
                'recommendationid': review['recommendationid'],
                'author_steamid': review['author']['steamid'],
                'review_text': review['review'],
                'timestamp_created': review['timestamp_created'],
                'timestamp_updated': review['timestamp_updated'],
                'voted_up': review['voted_up'],
                'votes_up': review['votes_up'],
                'votes_funny': review['votes_funny'],
                'weighted_vote_score': review['weighted_vote_score'],
                'steam_purchase': review['steam_purchase'],
                'received_for_free': review['received_for_free'],
                'written_during_early_access': review['written_during_early_access'],
            }
            selected_reviews.append(my_review_dict)

        # Get next page cursor
        cursor = reviews_response.get("cursor", "")
        if not cursor:
            print("Reached the last page!!!")
            break

        params["cursor"] = cursor
        print(f"Moving to page {page + 1}. Cursor: {cursor}")
        page += 1
        time.sleep(2)

    time.sleep(2)  # Wait before fetching the next app

# Save the reviews to a CSV file
if selected_reviews:
    reviews_df = pd.DataFrame(selected_reviews)
    csv_filename = f"reviews_{start_time.strftime('%Y%m%d-%H%M%S')}_{end_time.strftime('%Y%m%d-%H%M%S')}.csv"
    reviews_df.to_csv(csv_filename, index=False, encoding='utf-8')
    print(f"Reviews saved to {csv_filename}")
else:
    print("No reviews were collected.")

print("Finished processing all app reviews.")

Start time: 2024-01-01 00:00:00
End time: 2025-04-04 16:00:16.856347
1704056400.0 1743771616.856347

Getting reviews for app_id: 1245620
Getting reviews for app_id: 1245620 page: 0
Moving to page 1. Cursor: AoJwx8ut4ZUDeOXC2AU=
Getting reviews for app_id: 1245620 page: 1
Moving to page 2. Cursor: AoJwnoL4zZUDd8f51QU=
Getting reviews for app_id: 1245620 page: 2
Moving to page 3. Cursor: AoJw+aSvopUDfseq0AU=
Getting reviews for app_id: 1245620 page: 3
Moving to page 4. Cursor: AoJw+oLR+pQDev+CzAU=
Getting reviews for app_id: 1245620 page: 4
Moving to page 5. Cursor: AoJ4zeb21JQDfMPYxwU=
Getting reviews for app_id: 1245620 page: 5
Moving to page 6. Cursor: AoJ4pKPLvZQDdYiOxQU=
Getting reviews for app_id: 1245620 page: 6
Moving to page 7. Cursor: AoJwmP2nppQDc/a/wgU=
Getting reviews for app_id: 1245620 page: 7
Moving to page 8. Cursor: AoJ49deFlJQDepiAwAU=
Getting reviews for app_id: 1245620 page: 8
Moving to page 9. Cursor: AoJwhJ/6g5QDe8WzvQU=
Getting reviews for app_id: 1245620 page: 9
